In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [ ]:
df = pd.read_csv("transactions_train.csv", dtype={"article_id": str})
df_article = pd.read_csv("articles.csv", dtype={"article_id": str})

df = df.tail(2000000).reset_index(drop=True)
df["t_dat"] = pd.to_datetime(df["t_dat"])
df["week"] = (df["t_dat"].max() - df["t_dat"]).dt.days // 7

article_count = df["article_id"].value_counts()
df = df[df["article_id"].map(article_count) >= 50]
top_articles_df = df["article_id"].value_counts().head(100).reset_index()
top_articles_df.columns = ["article_id", "counts"]
df_article = df_article.merge(top_articles_df, on="article_id", how="right")
df_article = df_article.set_index("article_id")

top_articles = top_articles_df["article_id"]
df_article = df_article.loc[top_articles]
df_article[["prod_name", "product_code", "product_type_name", "counts", "perceived_colour_value_name"]]
df_article["product_type_name"].value_counts()


In [ ]:
WEEK_HIST_MAX = 5
df["article_id"] = pd.factorize(df['article_id'])[0]
article_ids = np.unique(df["article_id"].values)

def create_dataset(df, week):
    hist_df = df[(df["week"] > week) & (df["week"] <= week + WEEK_HIST_MAX)]
    hist_df = hist_df.groupby("customer_id").agg({"article_id": list}).reset_index()
    
    target_df = df[df["week"] == week]           
    target_df = target_df.groupby("customer_id").agg({"article_id": list}).reset_index()
    target_df.rename(columns={"article_id": "target"}, inplace=True)
    target_df["week"] = week
    target_df = target_df.merge(hist_df, on="customer_id", how="left")
    target_df = target_df[target_df["article_id"].notna()].reset_index(drop=True)
    target_df = target_df[target_df["target"].notna()].reset_index(drop=True)

    return target_df

test_weeks = [0]
train_weeks = [1, 2]


In [ ]:
a = create_dataset(df, 2)
a

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from tqdm import tqdm

In [ ]:
class Dataset(Dataset):
    def __init__(self, df, seq_len):
        self.df = df
        self.seq_len = seq_len
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        article_hist = torch.zeros(self.seq_len).long()
        target = torch.zeros(len(article_ids)).float()

        for t in row.target:
            target[t] = 1.0
                    
        if len(row.article_id) >= self.seq_len:
            article_hist = torch.LongTensor(row.article_id[-self.seq_len:])
        else:
            article_hist[-len(row.article_id):] = torch.LongTensor(row.article_id)
                
        return article_hist, target

In [ ]:
class NeuralAttentiveRecMachine(nn.Module):
    def __init__(self, article_dim):
        super(NeuralAttentiveRecMachine, self).__init__()
        
        self.article_emb = nn.Embedding(len(article_ids), article_dim)
        self.encoder = nn.GRU(article_dim, article_dim, batch_first=True)
        self.attention_layer = nn.Linear(article_dim, article_dim)
        self.softmax = nn.Softmax(dim=-1)
        self.context_layer = nn.Linear(2*article_dim, article_dim)
        
    def forward(self, article_hist):
        tensor = self.article_emb(article_hist)
        encoder_output, hn = self.encoder(tensor)
        hn = hn.squeeze(0)
        attention_logits = self.attention_layer(hn).unsqueeze(1) @ encoder_output.transpose(1, 2)
        attention_weights = self.softmax(attention_logits)
        context_vector = torch.concat([hn, (attention_weights @ encoder_output).squeeze(1)], -1)
        context_vector = self.context_layer(context_vector)
        article_embs = self.article_emb(torch.tensor(article_ids))
        score = context_vector @ article_embs.transpose(0, 1)
        
        return score
        
        

In [ ]:
class GruforRec(nn.Module):
    def __init__(self, article_dim):
        super(GruforRec, self).__init__()
        self.article_emb = nn.Embedding(len(article_ids), article_dim)
        self.encoder = nn.GRU(article_dim, article_dim, batch_first=True)
        
    def forward(self, article_hist):
        tensor = self.article_emb(article_hist)
        _, hn = self.encoder(tensor)
        hn = hn.squeeze(0)
        article_embs = self.article_emb(torch.tensor(article_ids))
        score = hn @ article_embs.transpose(0, 1)
        
        return score

In [ ]:
def eval_recall(output, target):
    sigmoid = nn.Sigmoid()
    output = sigmoid(output)
    recall = output > 0.4
    recall = ((recall == target.bool()) & recall).float().sum(dim=1) / (target.sum(dim=1) + 1e-10)
    recall = recall.sum(dim=0) / recall.shape[0]
    
    return recall

In [ ]:
def eval_precision(output, target):
    sigmoid = nn.Sigmoid()
    output = sigmoid(output)
    precision = output > 0.4
    precision = ((precision == target.bool()) & precision).float().sum(dim=1) / (precision.float().sum(dim=1) + 1e-10)
    precision = precision.sum(dim=0) / precision.shape[0]
    
    return precision

In [ ]:
SEQ_LEN = 16
BATCH_SIZE = 256
LEARNING_RATE = 0.001
EPOCHS = 30

test_df = pd.concat([create_dataset(df, w) for w in test_weeks]).reset_index(drop=True)
train_df = pd.concat([create_dataset(df, w) for w in train_weeks]).reset_index(drop=True)
train_dataset = Dataset(train_df, SEQ_LEN)
test_dataset = Dataset(test_df, SEQ_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model1 = NeuralAttentiveRecMachine(50)
model2 = GruforRec(50)


optimizer1 = torch.optim.SGD(model1.parameters(), lr=LEARNING_RATE)
optimizer2 = torch.optim.SGD(model2.parameters(), lr=LEARNING_RATE)

criterion = nn.BCEWithLogitsLoss()


In [ ]:

for i in range(EPOCHS):
    print("="*30)
    print(f"Current Epoch {i+1}")
    print("-"*30)
    print("Training..")
    
    _loss = .0
    train_recall = []
    train_precision = []
    model1.train()
    for article_hist, target in tqdm(train_loader):
        loss = criterion(model1(article_hist), target)
        _loss += loss.item()
        train_recall.append(eval_recall(model1(article_hist), target))
        train_precision.append(eval_precision(model1(article_hist), target))

        optimizer1.zero_grad()
        loss.backward()
        optimizer1.step()
    
    print("Evaluating..")
    _eval_loss = .0
    test_recall = []
    test_precision = []
    model1.eval()
    for article_hist, target in tqdm(test_loader):
        eval_loss = criterion(model1(article_hist), target)
        test_recall.append(eval_recall(model1(article_hist), target))
        test_precision.append(eval_precision(model1(article_hist), target))

        _eval_loss += eval_loss.item()
    
    train_loss = _loss/len(train_loader.dataset) * 100
    eval_loss = _eval_loss/len(test_loader.dataset) * 100
    train_recall = torch.mean(torch.tensor(train_recall))
    test_recall = torch.mean(torch.tensor(test_recall))
    train_precision = torch.mean(torch.tensor(train_precision))
    test_precision = torch.mean(torch.tensor(test_precision))
    print(f"Epoch {i+1}\nTrain Loss: {train_loss}\nTest Loss: {eval_loss}\nTrain Recall: {train_recall}\nTest Recall: {test_recall}\nTrain Precision: {train_precision}\nTest Precision: {test_precision}")

In [ ]:
for i in range(EPOCHS):
    print("="*30)
    print(f"Current Epoch {i+1}")
    print("-"*30)
    print("Training..")
    
    _loss = .0
    train_recall = []
    train_precision = []
    model2.train()
    for article_hist, target in tqdm(train_loader):
        loss = criterion(model2(article_hist), target)
        _loss += loss.item()
        train_recall.append(eval_recall(model2(article_hist), target))
        train_precision.append(eval_precision(model2(article_hist), target))

        optimizer2.zero_grad()
        loss.backward()
        optimizer2.step()
    
    print("Evaluating..")
    _eval_loss = .0
    test_recall = []
    test_precision = []
    model2.eval()
    for article_hist, target in tqdm(test_loader):
        eval_loss = criterion(model2(article_hist), target)
        test_recall.append(eval_recall(model2(article_hist), target))
        test_precision.append(eval_precision(model2(article_hist), target))

        _eval_loss += eval_loss.item()
    
    train_loss = _loss/len(train_loader.dataset) * 100
    eval_loss = _eval_loss/len(test_loader.dataset) * 100
    train_recall = torch.mean(torch.tensor(train_recall))
    test_recall = torch.mean(torch.tensor(test_recall))
    train_precision = torch.mean(torch.tensor(train_precision))
    test_precision = torch.mean(torch.tensor(test_precision))
    print(f"Epoch {i+1}\nTrain Loss: {train_loss}\nTest Loss: {eval_loss}\nTrain Recall: {train_recall}\nTest Recall: {test_recall}\nTrain Precision: {train_precision}\nTest Precision: {test_precision}")